In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, mean_squared_error

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
df = pd.read_excel(r"C:\Users\adishree.b\Documents\walmart.xlsx")

In [3]:
df.head()

,Converge,Campaign Name,Media Buy Name,Creative Name,Creative Image,Targeting Type,Creative Size,L1 Vertical,L2 Vertical,IO Start Date,IO End Date,Impressions,Clicks,Color,CTA,Product Shot / Product Portfolio,Lifestyle Imagery,Heavy Copy,Savings Messaging
0,01_ABB_J9273,01_ABB_J9273_AbbottMommy,BT_DO_P3_01_ABB_J9273_AbbottMommy,01_ABB_J9273_AbbottMommy_DIS_B_160x600_SkyScpr...,https://tpc.googlesyndication.com/pageadimg/im...,Demographic,160x600,NaN,NaN,2018-01-01,2018-02-28,202255.0,34.0,White,Shop Now,Product Portfolio,Yes,No,No
1,01_ABB_J9273,01_ABB_J9273_AbbottMommy,BT_ORGANIC_P3_01_ABB_J9273_AbbottMommy,01_ABB_J9273_AbbottMommy_DIS_B_160x600_SkyScpr...,https://tpc.googlesyndication.com/pageadimg/im...,Behavioral,160x600,NaN,NaN,2018-01-01,2018-02-28,176435.0,21.0,White,Shop Now,Product Portfolio,Yes,No,No
2,01_ABB_J9273,01_ABB_J9273_AbbottMommy,CT_BABY_P3_01_ABB_J9273_AbbottMommy,01_ABB_J9273_AbbottMommy_DIS_B_160x600_SkyScpr...,https://tpc.googlesyndication.com/pageadimg/im...,Contextual,160x600,NaN,NaN,2018-01-01,2018-02-28,86194.0,45.0,White,Shop Now,Product Portfolio,Yes,No,No
3,01_ABB_J9273,01_ABB_J9273_AbbottMommy,BT_BRAND_P3_01_ABB_J9273_AbbottMommy,01_ABB_J9273_AbbottMommy_DIS_B_160x600_SkyScpr...,https://tpc.googlesyndication.com/pageadimg/im...,Behavioral,160x600,NaN,NaN,2018-01-01,2018-02-28,60277.0,9.0,White,Shop Now,Product Portfolio,Yes,No,No
4,01_ABB_L6179,01_ABB_L6179_PediaSure18,BT_BRAND_01_ABB_L6179_PediaSure18,160x600_NEILSEN_BT_BRAND_01_ABB_L6179_PediaSur...,https://tpc.googlesyndication.com/pageadimg/im...,Behavioral,160x600,Baby & Toddlers,Infant Care & Consumables,2018-01-22,2018-04-22,156743.0,19.0,Yellow,Shop Now,Product Shot,Yes,No,No


In [4]:
df.columns

Index(['Converge', 'Campaign Name', 'Media Buy Name', 'Creative Name',
       'Creative Image', 'Targeting Type', 'Creative Size', 'L1 Vertical',
       'L2 Vertical', 'IO Start Date', 'IO End Date', 'Impressions', 'Clicks',
       'Color', 'CTA', 'Product Shot / Product Portfolio', 'Lifestyle Imagery',
       'Heavy Copy', 'Savings Messaging'],
      dtype='object')

In [5]:
df1 = df.iloc[:,5:9]

In [6]:
df1[['Impressions','Clicks']]= df[['Impressions','Clicks']]

In [7]:
df1.isnull().sum()

Targeting Type      0
Creative Size       0
L1 Vertical       546
L2 Vertical       546
Impressions         8
Clicks              8
dtype: int64

In [8]:
df1.fillna('Others', inplace = True)

In [9]:
df2 = df1[df1['Impressions']!='Others']

In [10]:
df1.shape

(12658, 6)

In [18]:
# Categorical boolean mask
categorical_feature_mask = df2.dtypes==object

# filter categorical columns using mask and turn it into a list
categorical_cols = df2.columns[categorical_feature_mask].tolist()


In [19]:
le = LabelEncoder()
# apply le on categorical feature columns
df2[categorical_cols] = df2[categorical_cols].apply(lambda col: le.fit_transform(col))
df2[categorical_cols].head(10)

C:\Users\adishree.b\AppData\Local\Continuum\anaconda3\envs\python3-TF2.0\lib\site-packages\pandas\core\frame.py:3489: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,Targeting Type,Creative Size,L1 Vertical,L2 Vertical,Impressions,Clicks
0,2,0,13,33,9222,34
1,0,0,13,33,9007,21
2,1,0,13,33,7666,45
3,0,0,13,33,6880,9
4,0,0,2,23,8851,19
5,0,0,2,23,8840,14
6,1,0,2,23,7219,30
7,1,0,2,23,7207,32
8,0,0,2,23,10502,41
9,0,0,2,23,10501,36


In [30]:
df2[categorical_cols].head()

,Targeting Type,Creative Size,L1 Vertical,L2 Vertical,Impressions,Clicks
0,2,0,13,33,9222,34
1,0,0,13,33,9007,21
2,1,0,13,33,7666,45
3,0,0,13,33,6880,9
4,0,0,2,23,8851,19


In [31]:
X_tot = df2[categorical_cols]
y = df2['Impressions']
y2 = df2['Clicks']/df2['Impressions']

In [38]:
X = X_tot.iloc[:,:4]

In [44]:
y2.dtypes

dtype('float64')

In [42]:
lm = LinearRegression()
scorer = make_scorer(mean_squared_error, greater_is_better=True)
scores = cross_val_score(lm,X,y,cv=10, scoring = scorer)
scores

array([1.86523445e-23, 1.29499288e-23, 4.79510448e-24, 3.67725333e-24,
       4.52648590e-23, 2.67177795e-24, 7.23011173e-24, 3.97472563e-23,
       5.23146977e-25, 2.76408293e-24])

In [43]:
rmse_scores = np.sqrt(scores)
rmse_scores

array([4.31883601e-12, 3.59860096e-12, 2.18977270e-12, 1.91761658e-12,
       6.72791639e-12, 1.63455742e-12, 2.68888671e-12, 6.30454251e-12,
       7.23289000e-13, 1.66255314e-12])

In [44]:
rmse_scores_mean = np.sqrt(cross_val_score(lm,X,y,cv=10, scoring = make_scorer(mean_squared_error))).mean()
rmse_scores_mean

3.1766571408845297e-12

## calssification

In [51]:
df2['ctr_group'] = np.where(df2['Clicks']/df2['Impressions']<=.001,'very low',
                               (np.where(df2['Clicks']/df2['Impressions']<=.003,'low',
                                         (np.where(df2['Clicks']/df2['Impressions']<=.005,'moderate',
                                                  (np.where(df2['Clicks']/df2['Impressions']<=.008,'good','high')))))))

C:\Users\adishree.b\AppData\Local\Continuum\anaconda3\envs\python3-TF2.0\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [52]:
df2['ctr_group'].value_counts()

very low    4237
low         3749
high        1964
moderate    1578
good        1122
Name: ctr_group, dtype: int64

In [41]:
# Combine the two dimensions of the input into one input matrix. 
# This is the X matrix from the linear model y = x*w + b.
# column_stack is a Numpy method, which combines two matrices (vectors) into one.
generated_inputs = X


# Produce the targets according to our f(x,z) = 2x - 3z + 5 + noise definition.
# In this way, we are basically saying: the weights should be 2 and -3, while the bias is 5.
generated_targets = y2

# save into an npz file called "TF_intro"
np.savez('TF_intro', inputs=generated_inputs, targets=generated_targets)

In [42]:
# Load the training data from the NPZ
training_data = np.load('TF_intro.npz')

In [45]:
# Declare a variable where we will store the input size of our model
# It should be equal to the number of variables you have
input_size = 4
# Declare the output size of the model
# It should be equal to the number of outputs you've got (for regressions that's usually 1)
output_size = 1

# Outline the model
# We lay out the model in 'Sequential'
# Note that there are no calculations involved - we are just describing our network
model = tf.keras.Sequential([
                            # Each 'layer' is listed here
                            # The method 'Dense' indicates, our mathematical operation to be (xw + b)
                            tf.keras.layers.Dense(output_size,
                                                 # there are extra arguments you can include to customize your model
                                                 # in our case we are just trying to create a solution that is 
                                                 # as close as possible to our NumPy model
                                                 kernel_initializer=tf.random_uniform_initializer(minval=-0.1, maxval=0.1),
                                                 bias_initializer=tf.random_uniform_initializer(minval=-0.1, maxval=0.1)
                                                 )
                            ])

# We can also define a custom optimizer, where we can specify the learning rate
custom_optimizer = tf.keras.optimizers.SGD(learning_rate=0.02)
# Note that sometimes you may also need a custom loss function 
# That's much harder to implement and won't be covered in this course though

# 'compile' is the place where you select and indicate the optimizers and the loss
model.compile(optimizer=custom_optimizer, loss='mean_squared_error')

# finally we fit the model, indicating the inputs and targets
# if they are not otherwise specified the number of epochs will be 1 (a single epoch of training), 
# so the number of epochs is 'kind of' mandatory, too
# we can play around with verbose; we prefer verbose=2
model.fit(training_data['inputs'], training_data['targets'], epochs=100, verbose=2)

TypeError: Unable to build `Dense` layer with non-floating point dtype <dtype: 'int32'>